## Set-up

### Imports

In [ ]:
%load_ext autoreload
%autoreload 2

import datetime
import glob
import ipdb
import matplotlib.pyplot as plt
plt.style.use('seaborn-colorblind')  # https://matplotlib.org/users/style_sheets.html
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import os
import pandas as pd
from pandas.plotting import scatter_matrix
import plotnine as gg
gg.theme_set(gg.theme_bw)
from scipy import stats
from sklearn import manifold, decomposition, preprocessing
from sklearn.linear_model import LogisticRegression
# Statsmodels.OLS ( )
import seaborn as sns
import time

from AlienTask import Task
from shared_aliens import update_Qs_sim, get_alien_paths, get_alien_initial_q,\
    get_summary_initial_learn, get_summary_cloudy, simulate_competition_phase, simulate_rainbow_phase, get_summary_rainbow,\
    read_in_human_data, se, get_chosen_TS

### Switches

In [ ]:
# Define things
do_isomap = False
do_save_selected_agents = True
if do_isomap:
    from sklearn import manifold, decomposition, preprocessing

# Which model will be simulated?
model_name = 'Bayes'
models = ['hier', 'Bayes', 'flat']
n_sim_per_subj, n_subj = 1, 26  # n_sim_per_sub = 20, n_subj = 31 (version3.1)  # TODO should be 1, 31-x (1 sim per person; exclude excluded subjects)
n_sim = n_sim_per_subj * n_subj
n_actions, n_aliens, n_seasons, n_TS = 3, 4, 3, 3
alien_initial_Q = get_alien_initial_q(model_name)

In [ ]:
# Data paths
## Where is human experimental data stored?
human_data_path = get_alien_paths()["human data prepr"]

## Where will simulated summaries be saved or read in?
summary_save_dir = os.path.join(get_alien_paths(False)['fitting results'], 'SummariesInsteadOfFitting_revision')
full_summary_save_dir = os.path.join(summary_save_dir, 'FullSimulations')

## Where will plots be saved that we create here?
plot_save_dir = os.path.join(summary_save_dir, 'plots')  # 'C:/Users/maria/MEGAsync/Berkeley/TaskSets/paperplots/'

In [ ]:
# Allowed parameter ranges to be sampled from
def get_param_names_ranges(model_name):
    
    if model_name == 'hier':
        param_names = ['alpha', 'beta', 'forget', 'alpha_high', 'beta_high', 'forget_high']
        param_ranges = pd.DataFrame.from_dict(
            {'alpha': [0, 1], 'beta': [1, 20], 'forget': [0, 1],
             'alpha_high': [0, 1], 'beta_high': [1, 20], 'forget_high': [0, 1]
             })

    elif model_name == 'flat':
        param_names = ['alpha', 'beta', 'forget']
        param_ranges = pd.DataFrame.from_dict({'alpha': [0, 1], 'beta': [1, 20], 'forget': [0, 1]})

    elif model_name == 'Bayes':
        param_names = ['alpha', 'beta', 'forget', 'beta_high', 'forget_high']
        param_ranges = pd.DataFrame.from_dict(
            {'alpha': [0, 1], 'beta': [1, 20], 'forget': [0, 1],
             'beta_high': [1, 20], 'forget_high': [0, 1]
             })

    else:
        raise(NameError, 'model_name must be "flat", "Bayes", or "hier"!')
        
    return param_names, param_ranges

# Example use
param_names, param_ranges = get_param_names_ranges(model_name)
param_names, param_ranges

In [ ]:
# Column names for the simulation summary measures
## Initial learning phase
IL_cols = ['IL_saving_av', 'IL_saving_first_trial', 'IL_saving_last_trial',  # savings
           'IL_acc_current_TS', 'IL_acc_prev_TS', 'IL_acc_other_TS',  # intrusion errors
           'IL_acc_current_TS_se', 'IL_acc_prev_TS_se', 'IL_acc_other_TS_se',
           'IL_perf_TS0', 'IL_perf_TS1', 'IL_perf_TS2',  # TS values
           'IL_perf_TS0_se', 'IL_perf_TS1_se', 'IL_perf_TS2_se', 'IL_perf_TS_corr'
           ]

## Cloudy phase
CL_cols = ['CL_acc_trial0', 'CL_acc_trial1', 'CL_acc_trial2', 'CL_acc_trial3',
           'CL_acc_trial0_se', 'CL_acc_trial1_se', 'CL_acc_trial2_se', 'CL_acc_trial3_se',
           'CL_slope', 'CL_slope_TS0', 'CL_slope_TS1', 'CL_slope_TS2']  # TS reactivation

## Competition phase
CO_cols = ['CO_acc_season', 'CO_acc_season_alien', 'CO_acc_season_se', 'CO_acc_season_alien_se']  # competition alien values & TS values

## Rainbow phase
RB_cols = ['RB_alien0_action0', 'RB_alien0_action1', 'RB_alien0_action2',
           'RB_alien1_action0', 'RB_alien1_action1', 'RB_alien1_action2',
           'RB_alien2_action0', 'RB_alien2_action1', 'RB_alien2_action2',
           'RB_alien3_action0', 'RB_alien3_action1', 'RB_alien3_action2']
RB_sum_cols = ['TS0', 'TS1', 'TS2', 'None', 'TS0_se', 'TS1_se', 'TS2_se', 'None_se']
summary_dat_cols = param_names + IL_cols + CL_cols + CO_cols + RB_cols

In [ ]:
# Create task, get numbers of trials for each phase
task = Task(n_subj)
n_trials, _, _, _, _ = task.get_trial_sequence(get_alien_paths(False)["human data prepr"],
                                         n_subj, n_sim_per_subj, range(n_subj),
                                         phases=("1InitialLearning", "2CloudySeason"))

n_trials_ = {'1InitialLearn': np.sum(task.phase == '1InitialLearning'),
             '2CloudySeason': np.sum(task.phase == '2CloudySeason'),
             '4RainbowSeason': 4 * n_aliens,
             '5Competition': 3}

trials = {'1InitialLearn': range(n_trials_['1InitialLearn']),
          '2CloudySeason': range(n_trials_['1InitialLearn'],
                                 n_trials_['1InitialLearn'] + n_trials_['2CloudySeason'])}

## Create summaries

In [ ]:
def get_action_values(seasons, aliens, TS):
    """
    Looks up objective action values and TS values for a numpy array of seasons and aliens, given a specific TS.
    """
    return np.max(TS[seasons, aliens], axis=1)

def get_TS_values(seasons, TS):
    """
    Looks up objective action values and TS values for a numpy array of seasons and aliens, given a specific TS.
    """
    return np.mean(np.max(TS[seasons], axis=2), axis=1)

# Example use
s, a = np.array([0, 1, 2]), np.array([0, 0, 0])
s, a = [0, 1, 2], [0, 0, 0]
get_action_values(s, a, task.TS), get_TS_values(s, task.TS)

In [ ]:
# Function to calculate summaries
def get_summary(parameters, param_ranges, n_sim, n_subj, model, summary_or_fulldata='summary'):

    # Get parmaeters
    ## Scale parameters correctly
    parameters = param_ranges.loc[0] + (param_ranges.loc[1] - param_ranges.loc[0]) * parameters

    beta_shape = (n_sim, 1)  # Q_low_sub.shape -> [n_subj, n_actions]
    beta_high_shape = (n_sim, 1)  # Q_high_sub.shape -> [n_subj, n_TS]
    forget_shape = (n_sim, 1, 1, 1)  # Q_low[0].shape -> [n_subj, n_TS, n_aliens, n_actions]
    forget_high_shape = (n_sim, 1, 1)  # -> [n_subj, n_seasons, n_TS]

    ## Parameters present in all models (flat RL, hier RL, Bayes)
    alpha = parameters['alpha'] * np.ones(n_sim)
    beta = parameters['beta'] * np.ones(beta_shape)
    forget = parameters['forget'] * np.ones(forget_shape)

    ## Deal with parameters that exist only in some models
    try:
        alpha_high = parameters['alpha_high'] * np.ones(n_sim)
    except KeyError:
        alpha_high = np.zeros(n_sim)
    try:
        beta_high = parameters['beta_high'] * np.ones(beta_high_shape)
    except KeyError:
        beta_high = np.zeros(beta_high_shape)
    try:
        forget_high = parameters['forget_high'] * np.ones(forget_high_shape)
    except KeyError:
        forget_high = np.zeros(forget_high_shape)
        
    # Initial learning phase
    ## Set up data storage
    seasons = np.zeros([n_trials, n_sim], dtype=int)
    corrects = np.zeros([n_trials, n_sim])
    rewards = np.zeros([n_trials, n_sim])
    aliens = np.zeros([n_trials, n_sim], dtype=int)
    actions = np.zeros([n_trials, n_sim], dtype=int)
    TSs = []

    ## Inialize Q-values
    Q_low = alien_initial_Q * np.ones([n_sim, n_TS, n_aliens, n_actions])
    Q_high = alien_initial_Q * np.ones([n_sim, n_seasons, n_TS])

    ## Simulate behavior
    for trial in trials['1InitialLearn']:

        ### Observe stimuli
        season, alien = task.present_stimulus(trial)

        ### Select action & update Q-values
        [Q_low, Q_high, TS, action, correct, reward, p_low] =\
            update_Qs_sim(season, alien,
                          Q_low, Q_high,
                          beta, beta_high, alpha, alpha_high, forget, forget_high,
                          n_sim, n_actions, n_TS, task, alien_initial_Q, model_name)
        
        ### Store trial data
        seasons[trial] = season
        corrects[trial] = correct
        rewards[trial] = reward
        aliens[trial] = alien
        actions[trial] = action
        TSs += [TS]

    ## Save final Q-values for subsequent phases
    final_Q_low = Q_low.copy()
    final_Q_high = Q_high.copy()
    ipdb.set_trace()

    ## Calculate summaries for initial learning phase
    summary_initial_learn = get_summary_initial_learn(
        seasons[trials['1InitialLearn']], corrects[trials['1InitialLearn']],
        aliens[trials['1InitialLearn']], actions[trials['1InitialLearn']],
        n_seasons, n_sim, trials, task)

    # Cloudy season
    cloudy_season = 0
    for trial in trials['2CloudySeason']:

        ## Observe trial stimuli
        old_season = season.copy()
        season, alien = task.present_stimulus(trial)

        ## Season switches
        if trial == list(trials['2CloudySeason'])[0]:
            season_switches = np.ones(n_sim, dtype=bool)
        else:
            season_switches = season != old_season
            
        ## Reset Q-values after each season switch to previously memorized values
        if (model_name == 'hier') or (model_name == 'Bayes'):
            Q_high_mem = np.max(final_Q_high, axis=1)  # [n_sim, TS] "memorized" correct TS in each season
            Q_high[season_switches] = alien_initial_Q  # not necessary, but looks cleaner
            Q_high[season_switches, cloudy_season] = Q_high_mem[season_switches]  # n_sim, n_seasons, n_TS

        elif model_name == 'flat':
            Q_low_mem = np.mean(final_Q_low, axis=1)  # [n_sim, n_aliens, n_actions] "memorized" overall task strategy
            Q_low[season_switches] = alien_initial_Q  # not necessary, but looks cleaner
            Q_low[season_switches, cloudy_season] = Q_low_mem[season_switches]
        
        else:
            raise(NameError, 'Model_name must be "flat", "hier", or "Bayes".')

        ## Update Q-values
        [Q_low, Q_high, TS, action, correct, reward, p_low] =\
            update_Qs_sim(cloudy_season * season, alien,
                          Q_low, Q_high,
                          beta, beta_high, alpha, alpha_high, forget, forget_high,
                          n_sim, n_actions, n_TS, task, alien_initial_Q, model_name)

        ## Store trial data
        seasons[trial] = season
        corrects[trial] = correct
        rewards[trial] = reward
        aliens[trial] = alien
        actions[trial] = action
        TSs += [TS]
    TSs = np.array(TSs)

    ## Calculate summaries for cloudy phase
    summary_cloudy = get_summary_cloudy(
        seasons[trials['2CloudySeason']], corrects[trials['2CloudySeason']],
        aliens[trials['2CloudySeason']], actions[trials['2CloudySeason']],
        n_sim, task)
    
    # Run and get summaries for competition phase
    comp_data = simulate_competition_phase(model_name, final_Q_high, final_Q_low, task,
                                           n_seasons, n_aliens, n_sim, beta_high, n_blocks_comp=n_trials_['5Competition'])
    summary_competition = comp_data.groupby('phase').aggregate('mean').reset_index()[['perc_selected_better', 'se']].T.values.flatten()
    summary_competition = pd.DataFrame(data=summary_competition, index=CO_cols)
    
    # Run and get summaries for rainbow season
    rainbow_dat = simulate_rainbow_phase(n_seasons, model_name, n_sim,
                                         beta, beta_high, final_Q_low, final_Q_high)
    rainbow_dat = pd.DataFrame(data=rainbow_dat.flatten(), index=RB_cols)
    
    # Run regression models for initial learn and cloudy
    regr_coefs = run_regr_models(
        seasons, corrects, aliens, actions, trials, regr_phases = ['1InitialLearn', '2CloudySeason'])

    # Return list of summaries
    if summary_or_fulldata == 'summary':
        return pd.concat(
            [parameters, summary_initial_learn, summary_cloudy, summary_competition, rainbow_dat, regr_coefs])
    elif summary_or_fulldata == 'fulldata':
        return seasons, corrects, rewards, aliens, actions, TSs
    else:
        raise ValueError('summary_or_fulldata must either be "summary" or "fulldata".')
        
# Example use
params = np.random.rand(len(param_names))
get_summary(params, param_ranges, n_sim, n_subj, model_name, summary_or_fulldata='fulldata')
get_summary(params, param_ranges, n_sim, n_subj, model_name, summary_or_fulldata='summary')

In [ ]:
def run_regr_models(seasons, corrects, aliens, actions, trials, regr_phases = ['1InitialLearn', '2CloudySeason']):
    
    """
    Run the regression correct ~ action_value + TS_value, separately for each participant
    Return mean and std over participants.
    """
    
    regr_coefs = []
    for phase in regr_phases:

        # Get data
        seasons_ph, corrects_ph, aliens_ph, actions_ph = seasons[
            trials[phase]], corrects[trials[phase]], aliens[trials[phase]], actions[trials[phase]]
        c = corrects_ph.astype(int)
        Qa = np.array([get_action_values(seasons_ph[trial], aliens_ph[trial], task.TS)
                       for trial in range(len(seasons_ph))])  # trial x subj
        Qts = np.array([get_TS_values(seasons_ph[trial], task.TS)
                        for trial in range(len(seasons_ph))])  # trial x subj

        # Run model for each agent
        coefs = []
        for subj in range(c.shape[1]):
            c_subj = c[:, subj]
            Qa_subj = Qa[:, subj]
            Qts_subj = Qts[:, subj]

            X_subj = np.array([Qa_subj, Qts_subj]).T
            y_subj = c[:, subj]

            X_subj, y_subj

            regr = LogisticRegression(solver='lbfgs')
            regr.fit(X_subj, y_subj)
            coefs += [regr.coef_.flatten()]

        coefs = pd.DataFrame(
            data=np.concatenate([np.mean(np.array(coefs), axis=0), np.std(np.array(coefs), axis=0)]),
            index=['{}_{}{}'.format(q, p, s) for q, p, s in zip(2 * ['Qa', 'Qts'], 4 * [phase], 2 * ['_mean'] + 2 * ['_se'])])
        regr_coefs += [coefs]

    return pd.concat(regr_coefs, axis=0)

# Example use
params = np.random.rand(len(param_names))
seasons, corrects, rewards, aliens, actions, TSs = get_summary(
            params, param_ranges, n_sim, n_subj, model_name, summary_or_fulldata='fulldata')
run_regr_models(seasons, corrects, aliens, actions, trials, regr_phases = ['1InitialLearn', '2CloudySeason'])

In [ ]:
save_frequency = 20
print_frequency = 20
n_frames = 100
n_sims_per_frame = 1000

# Get summaries for different parameters
new_time = time.time()

for frame_i in range(n_frames):
    summaries = pd.DataFrame()

    for sim_i in range(n_sims_per_frame):
        params = np.random.rand(len(param_names))
        new_summary = get_summary(
            params, param_ranges, n_sim, n_subj, model_name, summary_or_fulldata='summary')
        summaries = pd.concat([summaries, new_summary], axis=1)

        if (sim_i % print_frequency) == 0:

            # Print progress
            print("\n\tIteration {}".format(sim_i))
            old_time = new_time
            new_time = time.time()
            diff = new_time - old_time
            print("Time passed: {} seconds".format(np.round(diff)))

        if ((sim_i % save_frequency) == 0) or (sim_i == n_sims_per_frame - 1):

            # Save summaries to disk
            save_path = os.path.join(summary_save_dir, model_name + '_summaries_{}TEST.csv'.format(frame_i))
            save_summaries = summaries.T
            save_summaries['model'] = model_name
            save_summaries.to_csv(save_path)
            print("Saving summaries to {}".format(save_path))

## Read in simulation summaries and vizualize as density plots

In [ ]:
# Get human data
n_hum, hum_aliens, hum_seasons, hum_corrects, hum_actions, hum_rewards, hum_rainbow_dat, hum_comp_dat = read_in_human_data(
    human_data_path, 828, n_aliens, n_actions, exclude=['160',])  # '164' -> already excluded for head trauma

In [ ]:
# Get human initial learn data
hum_summary_initial_learn = get_summary_initial_learn(
    hum_seasons[trials['1InitialLearn']], hum_corrects[trials['1InitialLearn']],
    hum_aliens[trials['1InitialLearn']], hum_actions[trials['1InitialLearn']],
    n_seasons, n_hum, trials, task).T

hum_summary_initial_learn['IL_saving_last_minus_first'] = hum_summary_initial_learn['IL_saving_last_trial'] - hum_summary_initial_learn['IL_saving_first_trial']
hum_summary_initial_learn['IL_perf_TS0_minus_TS1'] = hum_summary_initial_learn['IL_perf_TS0'] - hum_summary_initial_learn['IL_perf_TS1']
hum_summary_initial_learn['IL_perf_TS0_minus_TS2'] = hum_summary_initial_learn['IL_perf_TS0'] - hum_summary_initial_learn['IL_perf_TS2']
hum_summary_initial_learn['IL_perf_TS1_minus_TS2'] = hum_summary_initial_learn['IL_perf_TS1'] - hum_summary_initial_learn['IL_perf_TS2']

hum_summary_initial_learn['IL_first_TS0_minus_TS2'] = hum_summary_initial_learn['IL_first_TS0'] - hum_summary_initial_learn['IL_first_TS2']
hum_summary_initial_learn['IL_first_TS0_minus_TS1'] = hum_summary_initial_learn['IL_first_TS0'] - hum_summary_initial_learn['IL_first_TS1']
hum_summary_initial_learn['IL_first_TS1_minus_TS2'] = hum_summary_initial_learn['IL_first_TS1'] - hum_summary_initial_learn['IL_first_TS2']

# Get human cloudy data
hum_summary_cloudy = get_summary_cloudy(
    hum_seasons[trials['2CloudySeason']], hum_corrects[trials['2CloudySeason']],
    hum_aliens[trials['2CloudySeason']], hum_actions[trials['2CloudySeason']],
    n_hum, task).T
hum_summary_cloudy['CL_first_TS0_minus_TS2'] = hum_summary_cloudy['CL_first_TS0'] - hum_summary_cloudy['CL_first_TS2']
hum_summary_cloudy['CL_first_TS0_minus_TS1'] = hum_summary_cloudy['CL_first_TS0'] - hum_summary_cloudy['CL_first_TS1']
hum_summary_cloudy['CL_first_TS1_minus_TS2'] = hum_summary_cloudy['CL_first_TS1'] - hum_summary_cloudy['CL_first_TS2']

hum_summary_cloudy['CL_slope_TS0minusTS2'] = hum_summary_cloudy['CL_slope_TS0'] - hum_summary_cloudy['CL_slope_TS2']
hum_summary_cloudy['CL_slope_TS0minusTS1'] = hum_summary_cloudy['CL_slope_TS0'] - hum_summary_cloudy['CL_slope_TS1']
hum_summary_cloudy['CL_slope_TS1minusTS2'] = hum_summary_cloudy['CL_slope_TS1'] - hum_summary_cloudy['CL_slope_TS2']

# Get human competition data
season_cols = [col for col in hum_comp_dat.columns.values if col[0] == "("]
alien_cols = [col for col in hum_comp_dat.columns.values if col[0] != "("]
season_perf = np.mean(hum_comp_dat[season_cols], axis=1)
alien_perf = np.mean(hum_comp_dat[alien_cols], axis=1)
season_mean = np.mean(season_perf)
season_se = np.std(season_perf) / np.sqrt(n_hum)
alien_mean = np.mean(alien_perf)
alien_se = np.std(alien_perf) / np.sqrt(n_hum)
comp_t, comp_p = stats.ttest_rel(season_perf, alien_perf)
hum_summary_competition = pd.DataFrame(np.array([[season_mean, alien_mean, season_se, alien_se, np.mean(season_perf-alien_perf)]]),
                                       columns=CO_cols + ['CO_season_minus_alien'])

# Get human rainbow data
# hum_rainbow_dat = pd.DataFrame(np.expand_dims(hum_rainbow_dat.flatten(), axis=0), columns=RB_cols)
hum_summary_rainbow = get_summary_rainbow(
    n_aliens, n_seasons, hum_rainbow_dat, task)
hum_summary_rainbow = pd.DataFrame(
    data=np.expand_dims(hum_summary_rainbow, axis=0),
    columns=RB_sum_cols)
hum_summary_rainbow['TS0_minus_TS1'] = hum_summary_rainbow['TS0'] - hum_summary_rainbow['TS1']
hum_summary_rainbow['TS0_minus_TS2'] = hum_summary_rainbow['TS0'] - hum_summary_rainbow['TS2']
hum_summary_rainbow['TS1_minus_TS2'] = hum_summary_rainbow['TS1'] - hum_summary_rainbow['TS2']

# Get regression models
hum_regr_coefs = run_regr_models(
    hum_seasons, hum_corrects, hum_aliens, hum_actions, trials, regr_phases = ['1InitialLearn', '2CloudySeason']).T
for phase in ['1InitialLearn', '2CloudySeason']:
    hum_regr_coefs['Qts_minus_a_{}'.format(phase)] = \
        hum_regr_coefs['Qts_{}_mean'.format(phase)] - hum_regr_coefs['Qa_{}_mean'.format(phase)]

In [ ]:
# Get agent summary filenames
filenames = glob.glob(os.path.join(summary_save_dir, '*.csv'))
filenames = [filename for filename in filenames if 'summar' in filename]  # don't read in selected_agents.csv etc.
print('Found {} files.'.format(len(filenames)))

In [ ]:
# Read in all csv with summaries
all_summaries = pd.DataFrame(columns=param_names)
for filename in filenames:
    summaries = pd.read_csv(filename, index_col=0)
    summaries = summaries.dropna()  # remove empty rows
    all_summaries = all_summaries.append(summaries)
all_summaries = all_summaries.reset_index(drop=True)

# Add other measures
all_summaries['IL_saving_last_minus_first'] = all_summaries['IL_saving_last_trial'] - all_summaries['IL_saving_first_trial']
all_summaries['IL_perf_TS0_minus_TS1'] = all_summaries['IL_perf_TS0'] - all_summaries['IL_perf_TS1']
all_summaries['IL_perf_TS0_minus_TS2'] = all_summaries['IL_perf_TS0'] - all_summaries['IL_perf_TS2']
all_summaries['IL_perf_TS1_minus_TS2'] = all_summaries['IL_perf_TS1'] - all_summaries['IL_perf_TS2']

all_summaries['CO_season_minus_alien'] = all_summaries['CO_acc_season'] - all_summaries['CO_acc_season_alien']
all_summaries['IL_first_TS0_minus_TS2'] = all_summaries['IL_first_TS0'] - all_summaries['IL_first_TS2']
all_summaries['IL_first_TS0_minus_TS1'] = all_summaries['IL_first_TS0'] - all_summaries['IL_first_TS1']
all_summaries['IL_first_TS1_minus_TS2'] = all_summaries['IL_first_TS1'] - all_summaries['IL_first_TS2']

all_summaries['CL_first_TS0_minus_TS2'] = all_summaries['CL_first_TS0'] - all_summaries['CL_first_TS2']
all_summaries['CL_first_TS0_minus_TS1'] = all_summaries['CL_first_TS0'] - all_summaries['CL_first_TS1']
all_summaries['CL_first_TS1_minus_TS2'] = all_summaries['CL_first_TS1'] - all_summaries['CL_first_TS2']

all_summaries['CL_slope_TS0minusTS2'] = all_summaries['CL_slope_TS0'] - all_summaries['CL_slope_TS2']
all_summaries['CL_slope_TS0minusTS1'] = all_summaries['CL_slope_TS0'] - all_summaries['CL_slope_TS1']
all_summaries['CL_slope_TS1minusTS2'] = all_summaries['CL_slope_TS1'] - all_summaries['CL_slope_TS2']

for phase in ['1InitialLearn', '2CloudySeason']:
    all_summaries['Qts_minus_a_{}'.format(phase)] = \
        all_summaries['Qts_{}_mean'.format(phase)] - all_summaries['Qa_{}_mean'.format(phase)]

print("Number of samples: {} (flat: {}; hier: {}; Bayes: {})".
      format(all_summaries.shape[0],
             np.sum(all_summaries['model'] == 'flat'),
             np.sum(all_summaries['model'] == 'hier'),
             np.sum(all_summaries['model'] == 'Bayes')
))

In [ ]:
# Get rainbow summary
summary_rainbow = pd.DataFrame()
for model in models:
    rainbow_dat = all_summaries.loc[all_summaries['model'] == model, RB_cols]
    rainbow_dat = rainbow_dat.values.reshape((rainbow_dat.shape[0], n_aliens, n_actions))
    
    summary_rainbow_mod = np.array([get_summary_rainbow(n_aliens, n_seasons, dat, task) for dat in rainbow_dat])
    summary_rainbow_mod = pd.DataFrame(summary_rainbow_mod, columns=RB_sum_cols)
    summary_rainbow_mod.loc[:, 'model'] = model
    
    cor = np.array([np.corrcoef(hum_rainbow_dat[0].flatten(), dat.flatten())[0, 1] for dat in rainbow_dat.astype(float)])
    summary_rainbow_mod.loc[:, 'corr_with_humans'] = cor
    
    summary_rainbow = summary_rainbow.append(summary_rainbow_mod)
    
summary_rainbow['TS0_minus_TS1'] = summary_rainbow['TS0'] - summary_rainbow['TS1']
summary_rainbow['TS0_minus_TS2'] = summary_rainbow['TS0'] - summary_rainbow['TS2']
summary_rainbow['TS1_minus_TS2'] = summary_rainbow['TS1'] - summary_rainbow['TS2']
summary_rainbow.head()

In [ ]:
# Plot correlations and histograms
for model_name in models:
    model_summaries = all_summaries.loc[all_summaries['model'] == model_name]
    model_summaries = model_summaries.reset_index(drop=True)
    pd.plotting.scatter_matrix(model_summaries.loc[:1000, ['alpha', 'beta', 'forget']])

In [ ]:
def get_means_sds(sim_dat, hum_dat, column, t_compare_value=0):
    
    means_sds = sim_dat.groupby("model")[column].agg(
        {'mean': 'mean',
         'std': 'std',
         'lik': lambda x: np.mean(x > hum_dat[column][0]),
         'lik2': lambda x: np.mean(x < hum_dat[column][0])})
    
    ttests = sim_dat.groupby('model')[column].agg(stats.ttest_1samp, t_compare_value)
    
    return means_sds, ttests

In [ ]:
def get_difference_data(dat1, dat2, n_bins=50):
    
    dat_both = np.append(dat1, dat2)
    
    min_bin, max_bin = min(dat_both), max(dat_both)
    
    hist1, bins1 = np.histogram(dat1, range=(min_bin, max_bin), bins=n_bins)
    hist2, bins2 = np.histogram(dat2, range=(min_bin, max_bin), bins=n_bins)
    assert np.all(bins1 == bins2)
    
    diff = hist2 - hist1
    
    diff_dat = pd.DataFrame({'diff': diff,
                             'bin': bins1[:-1]})

    return diff_dat

# Example usage
nbins = 50
phase = '1InitialLearn'

dat_flat = all_summaries.loc[all_summaries.model=='flat', 'Qts_{}_mean'.format(phase)].values
dat_hier = all_summaries.loc[all_summaries.model=='hier', 'Qts_{}_mean'.format(phase)].values

get_difference_data(dat_flat, dat_hier)

In [ ]:
# Reusable histogram function for all histogram plots
def make_histogram(sim_dat, hum_dat, col_name, figure_size=(5, 2),
                   xlim=None, ylim=None, vline=0, plot_name='test.png'):
    
    gg.options.figure_size = figure_size
    g = (gg.ggplot(sim_dat, gg.aes(col_name, color='model'))
     + gg.geom_density()
     + gg.xlab(col_name)
     + gg.scale_color_manual(['green', 'orange', 'blue'])
     + gg.geom_vline(xintercept=hum_dat[col_name], color='red')
     + gg.geom_vline(xintercept=vline, linetype='dotted')
     + gg.coord_cartesian(xlim=xlim, ylim=ylim)
    )
    save_path = os.path.join(plot_save_dir, plot_name)
    print("Saving to {}.".format(save_path))
    g.draw()
    g.save(save_path)

# Example use
plot_name_base = '0TS_react_hist'
make_histogram(all_summaries, hum_summary_cloudy, CL_cols[8], xlim=(-0.1, 0.25))

In [ ]:
def make_diff_hist(sim_dat, hum_dat, col_name, model_names, figure_size = (4, 3),
                   plot_name='test.png', n_bins=40, vline=0, xlim=None, ylim=None):

    all_diff_dat = pd.DataFrame()

    for model1_name, model2_name in model_names:
        dat1 = sim_dat.loc[sim_dat.model==model1_name, col_name].values
        dat2 = sim_dat.loc[sim_dat.model==model2_name, col_name].values

        diff_dat = get_difference_data(dat1, dat2, n_bins=n_bins)
        diff_dat['measure'] = col_name
        diff_dat['models'] = ' minus '.join([model2_name, model1_name])

        all_diff_dat = all_diff_dat.append(diff_dat)

    gg.options.figure_size = figure_size
    g = (gg.ggplot(all_diff_dat, gg.aes('bin', 'diff', color='models'))
     + gg.geom_line()
     + gg.xlab(col_name)
     + gg.ylab('Difference (# samples)')
     + gg.scale_color_manual(['green', 'orange'])
     + gg.geom_vline(xintercept=hum_dat[col_name], color='red')
     + gg.geom_vline(xintercept=vline, linetype='dotted')
     + gg.coord_cartesian(xlim=xlim, ylim=ylim)
    )
    save_path = os.path.join(plot_save_dir, plot_name)
    print("Saving to {}.".format(save_path))
    g.draw()
    g.save(save_path)
    
# Example use
model_names = [['flat', 'hier'], ['Bayes', 'hier']]
col_name = 'Qts_1InitialLearn_mean'
make_diff_hist(all_summarise, hum_summary_initial_learn, col_name, model_names, xlim=(-0.3, 0.3))

In [ ]:
# Which TS are used initially (first four trials, initial learning phase)?
plot_name_base = '2TS_values_regression'

for phase in ['1InitialLearn', '2CloudySeason']:
    cols = ['Qts_{}_mean'.format(phase), 'Qa_{}_mean'.format(phase)]

    for colname in cols:
        xlim = (-0.1, 0.25)
        make_histogram(all_summaries, hum_regr_coefs,
                       colname, xlim=xlim, ylim=(0, 15),
                       plot_name='{}_{}_hist.png'.format(plot_name_base, colname))

        make_diff_hist(all_summaries, hum_regr_coefs, colname, model_names, xlim=xlim,
                       plot_name='{}_{}_diff.png'.format(plot_name_base, colname))

In [ ]:
# Reactivation of TS (cloudy)
plot_name_base = '0TS_react_hist'

for colname in CL_cols[8:] + ['CL_slope_TS1minusTS2', 'CL_slope_TS0minusTS2', 'CL_slope_TS0minusTS1']:
    xlim = (-0.2, 0.4)
    make_histogram(all_summaries, hum_summary_cloudy, colname,
                   plot_name='{}_{}_hist.png'.format(plot_name_base, colname),
                   xlim=xlim)

    make_diff_hist(all_summaries, hum_summary_cloudy, colname, model_names, n_bins=18,
                   plot_name='{}_{}_diff.png'.format(plot_name_base, colname),
                   xlim=xlim)
    
# print(get_means_sds(all_summaries, hum_summary_cloudy, 'CL_slope'))
# print(get_means_sds(all_summaries, hum_summary_cloudy, 'CL_slope_TS1minusTS2'))
# print(get_means_sds(all_summaries, hum_summary_cloudy, 'CL_slope_TS0minusTS2'))
# print(get_means_sds(all_summaries, hum_summary_cloudy, 'CL_slope_TS0minusTS1'))

In [ ]:
# Intrusion errors (init. learn.)
plot_name_base = '1intrusion_errors_hist'

for colname in IL_cols[3:5]:
    xlim = (0.2, 0.5)
    make_histogram(all_summaries, hum_summary_initial_learn, colname,
                   plot_name='{}_{}_hist.png'.format(plot_name_base, colname),
                   xlim=xlim, vline=1/3)

    make_diff_hist(all_summaries, hum_summary_initial_learn, colname, model_names, n_bins=20,
                   plot_name='{}_{}_diff.png'.format(plot_name_base, colname),
                   xlim=xlim, vline=1/3)

# print(get_means_sds(all_summaries, hum_summary_initial_learn, IL_cols[3], 1/3))
# print(get_means_sds(all_summaries, hum_summary_initial_learn, IL_cols[4], 1/3))

In [ ]:
# TS values affect preference (competition)
plot_name_base = '3TS_values_preference_hist'

for colname in CO_cols[:2]:
    xlim = (0.45, 0.7)
    make_histogram(all_summaries, hum_summary_competition, colname,
                   plot_name='{}_{}_hist.png'.format(plot_name_base, colname),
                   xlim=xlim, vline=1/2)
    
    make_diff_hist(all_summaries, hum_summary_competition, colname, model_names,
                   plot_name='{}_{}_diff.png'.format(plot_name_base, colname),
                   xlim=xlim, vline=1/2)

colname = 'CO_season_minus_alien'
xlim = (-0.1, 0.1)
make_histogram(all_summaries, hum_summary_competition, colname,
               plot_name='{}_{}_hist.png'.format(plot_name_base, col_name),
               xlim=xlim)

make_diff_hist(all_summaries, hum_summary_competition, colname, model_names,
               plot_name='{}_{}_diff.png'.format(plot_name_base, col_name),
               xlim=xlim)

# get_means_sds(all_summaries, hum_summary_competition, 'CO_season_minus_alien')

In [ ]:
# TS values affect generalization (rainbow)
plot_name_base = '2TS_rainbow'

for colname in ['TS0_minus_TS2', 'TS0_minus_TS1', 'TS1_minus_TS2']:
    xlim = (-0.02, 0.15)
    make_histogram(summary_rainbow, hum_summary_rainbow, colname,
                   plot_name='{}_{}_hist.png'.format(plot_name_base, colname),
                   xlim=xlim, ylim=(0, 30))

    make_diff_hist(summary_rainbow, hum_summary_rainbow, colname, model_names,
                   plot_name='{}_{}_diff.png'.format(plot_name_base, colname), xlim=xlim)
    
# print(get_means_sds(summary_rainbow, hum_summary_rainbow, 'TS0_minus_TS2'))
# print(get_means_sds(summary_rainbow, hum_summary_rainbow, 'TS0_minus_TS1'))
# print(get_means_sds(summary_rainbow, hum_summary_rainbow, 'TS1_minus_TS2'))

In [ ]:
plot_name_base = '4TS_values'
for colname in ['IL_first_TS0_minus_TS2', 'IL_first_TS0_minus_TS1', 'IL_first_TS1_minus_TS2']:
    xlim = (-0.01, 0.15)
    make_histogram(all_summaries, hum_summary_initial_learn, colname,
                   plot_name='{}_{}_hist.png'.format(plot_name_base, colname),
                   xlim=xlim)
    
    make_diff_hist(all_summaries, hum_summary_initial_learn, colname, model_names, n_bins=15,
                   plot_name='{}_{}_diff.png'.format(plot_name_base, colname),
                   xlim=xlim)

In [ ]:
plot_name_base = '5TS_values'
for colname in ['CL_first_TS0_minus_TS2', 'CL_first_TS0_minus_TS1', 'CL_first_TS1_minus_TS2']:
    xlim = (-0.01, 0.15)
    make_histogram(all_summaries, hum_summary_cloudy, colname,
                   plot_name='{}_{}_hist.png'.format(plot_name_base, colname),
                   xlim=xlim)
    
    make_diff_hist(all_summaries, hum_summary_cloudy, colname, model_names, n_bins=15,
                   plot_name='{}_{}_diff.png'.format(plot_name_base, colname),
                   xlim=xlim)

In [ ]:
# TS values affect generalization (rainbow)
fig, axes = plt.subplots(nrows=len(models), figsize=(8, 8))
colors = sns.cubehelix_palette(4, start=.5, rot=-.75, reverse=True)[:-1]

for i, model in enumerate(models):
    dat = summary_rainbow.loc[summary_rainbow['model'] == model]

    # Plot
    for j, effect in enumerate(RB_sum_cols[:3]):
        sns.distplot(dat[effect], kde=True, hist=False, label=effect, color=colors[j], ax=axes[i])
        [ax.axvline(x=10/12/3, color='grey', linestyle='--') for ax in axes[:2]]
        [ax.set_xlim(0, 0.5) for ax in axes[:2]]
        [ax.set_ylim(0, 100) for ax in axes[:2]]
        axes[i].set_title(model)
        [ax.set_xlabel("") for ax in axes]
        axes[i].legend()

[ax.set_ylabel("Density") for ax in axes]
plt.tight_layout()
plt.savefig(plot_save_dir + '/4TS_values_generalization_hist2.png')
# get_means_sds(summary_rainbow, hum_summary_rainbow, 'TS0_minus_TS2')

In [ ]:
# Rainbow phase TS choices
plt.figure()
for model in models:

    # Get summary_rainbow
    rainbow_dat = all_summaries.loc[all_summaries['model'] == model, RB_cols]
    rainbow_dat = rainbow_dat.values.reshape((rainbow_dat.shape[0], n_aliens, n_actions))
    summary_rainbow = np.array([get_summary_rainbow(n_aliens, n_seasons, dat, task) for dat in rainbow_dat])
    summary_rainbow = pd.DataFrame(summary_rainbow, columns=RB_sum_cols)

    # Plot
    for i, effect in enumerate(RB_sum_cols[:3]):
        plt.subplot(2, 3, i+1)
        sns.distplot(summary_rainbow[effect], kde=True, hist=True, label=model)
        plt.axvline(x=10/12/3, color='grey', linestyle='--')
        plt.axvline(x=hum_summary_rainbow[effect].values, color='red', linestyle='-')
        plt.xlim(0, 0.5)
        plt.ylim(0, 60)
        plt.xlabel(effect)
        plt.ylabel("Density")

    effect = RB_sum_cols[3]
    plt.subplot(2, 3, 4)
    sns.distplot(summary_rainbow[effect], kde=True, hist=True, label=model)
    plt.axvline(x=(2/12), color='grey', linestyle='--')
    plt.axvline(x=hum_summary_rainbow[effect].values, color='red', linestyle='-')
    plt.xlim(0, 0.5)
    plt.ylim(0, 60)
    plt.xlabel(effect)
    plt.ylabel("Density")

    effect = 'TS0_minus_TS2'
    plt.subplot(2, 3, 5)
    sns.distplot(summary_rainbow[effect], kde=True, hist=True, label=model)
    plt.axvline(x=0, color='grey', linestyle='--')
    plt.axvline(x=hum_summary_rainbow[effect].values, color='red', linestyle='-')
    plt.xlim(-0.3, 0.3)
    plt.ylim(0, 60)
    plt.xlabel(effect)
    plt.ylabel("Density")

plt.legend()
plt.tight_layout()

In [ ]:
# Savings (init. learn.)
make_histogram(all_summaries,
               hum_summary_initial_learn, IL_cols[:3] + ['IL_saving_last_minus_first'],
               plot_name='5Savings_hist.png')

In [ ]:
# Plot paramter - marker correlations
hum_sums = hum_summary_initial_learn[['IL_acc_prev_TS', 'IL_acc_current_TS', 'IL_perf_TS0_minus_TS1']]
hum_sums['CL_slope'] = hum_summary_cloudy['CL_slope']
hum_sums['CO_season_minus_alien'] = hum_summary_competition['CO_season_minus_alien']
markers = ['CL_slope', 'IL_acc_current_TS', 'IL_acc_prev_TS', 'IL_perf_TS0_minus_TS1', 'CO_season_minus_alien']

for model, params in zip(models, [param_names[:3], param_names]):

    # Correlations between markers and parameters
    dat = all_summaries.loc[all_summaries['model'] == model]
    sns.pairplot(dat, x_vars=params, y_vars=markers, kind='reg',
             plot_kws={'color': 'grey', 'fit_reg': False, 'scatter_kws': {'s': 1}})
    plt.savefig(os.path.join(plot_save_dir, '/CorrMarkerParam_{}.png'.format(model))

    # Correlations between different markers
    sns.pairplot(dat, vars=markers, kind='reg',
                 plot_kws={'color': 'grey', 'fit_reg': False, 'scatter_kws': {'s': 1}},
                 diag_kind='kde', diag_kws={'color': 'grey', 'shade': True})
    # add human data
    # plt.scatter(hum_sums)
    plt.savefig(os.path.join(plot_save_dir, '/CorrParamParam_{}.png'.format(model))

# Find good simulations

In [ ]:
def get_selected_agents(width):
    lower = (10 - width/2) / 10
    upper = (10 + width/2) / 10

    # Get masks for each effects
    IL_acc_prev_TS = (
            all_summaries['IL_acc_prev_TS'] > lower * hum_summary_initial_learn['IL_acc_prev_TS'].values[0]) & (
            all_summaries['IL_acc_prev_TS'] < upper * hum_summary_initial_learn['IL_acc_prev_TS'].values[0])
    IL_acc_current_TS = (
            all_summaries['IL_acc_current_TS'] > lower * hum_summary_initial_learn['IL_acc_current_TS'].values[0]) & (
            all_summaries['IL_acc_current_TS'] < upper * hum_summary_initial_learn['IL_acc_current_TS'].values[0])
    CO_acc_season = (
            all_summaries['CO_acc_season'] > lower * hum_summary_competition['CO_acc_season'].values[0]) & (
            all_summaries['CO_acc_season'] < upper * hum_summary_competition['CO_acc_season'].values[0])
    CO_acc_season_alien = (
            all_summaries['CO_acc_season_alien'] > lower * hum_summary_competition['CO_acc_season_alien'].values[0]) & (
            all_summaries['CO_acc_season_alien'] < upper * hum_summary_competition['CO_acc_season_alien'].values[0])
    CL_slope = (
            all_summaries['CL_slope'] > lower * hum_summary_cloudy['CL_slope'].values[0]) & (
            all_summaries['CL_slope'] < upper * hum_summary_cloudy['CL_slope'].values[0])
    Qts_1InitialLearn_mean = (
            all_summaries['Qts_1InitialLearn_mean'] > lower * hum_regr_coefs['Qts_1InitialLearn_mean'].values[0]) & (
            all_summaries['Qts_1InitialLearn_mean'] < upper * hum_regr_coefs['Qts_1InitialLearn_mean'].values[0])
    Qa_1InitialLearn_mean = (
            all_summaries['Qa_1InitialLearn_mean'] > lower * hum_regr_coefs['Qa_1InitialLearn_mean'].values[0]) & (
            all_summaries['Qa_1InitialLearn_mean'] < upper * hum_regr_coefs['Qa_1InitialLearn_mean'].values[0])

    # Subset data
    selected_agents = all_summaries.loc[
        IL_acc_prev_TS & IL_acc_current_TS & CO_acc_season & CO_acc_season_alien & CL_slope
        & Qts_1InitialLearn_mean & Qa_1InitialLearn_mean
    ]  # IL_acc_prev_TS & CL_slope & CO_season_minus_alien & IL_perf_TS2minus1

    # Count models
    model_ns = [selected_agents.loc[selected_agents['model'] == model,].shape[0] for model in models]

    return selected_agents, model_ns

# Example use
get_selected_agents(6)[0]

In [ ]:
# How many models do we get depending on width of interval?
widths = np.arange(0, 50)

all_ns = []
for width in widths:
    selected_agents, model_ns = get_selected_agents(width)
    all_ns += [model_ns + [lower, upper, width]]

all_ns = pd.DataFrame(data=all_ns, columns=['n_{}'.format(m) for m in models] + ['lower', 'upper', 'width'])

In [ ]:
all_ns_long = pd.melt(all_ns, id_vars=('lower', 'upper', 'width'), var_name='model', value_name='n')
all_ns_long['log_n'] = np.log(all_ns_long['n'])

print(gg.ggplot(all_ns_long, gg.aes('width', 'n', color='model'))
 + gg.geom_point()
 + gg.geom_vline(xintercept=10)
)
print(gg.ggplot(all_ns_long, gg.aes('width', 'log_n', color='model'))
 + gg.geom_point()
 + gg.geom_vline(xintercept=10)
)

In [ ]:
# Select agents for one width
selected_agents, model_ns = get_selected_agents(width=10)
print("Found {} agents: {}, {}".format(selected_agents.shape[0], models, model_ns))

# Save selected_agents as csv
if do_save_selected_agents:
    save_path = summary_save_dir + '/selected_agents.csv'
    print("Saving selected_agents to {}".format(save_path))
    selected_agents.to_csv(save_path, index=False)

for model in ['hier', 'flat']:
    sub_dat = selected_agents.loc[selected_agents['model'] == model]
    sub_dat[['beta', 'beta_high']] /= 20
    if model == 'flat':
        param_names_ = param_names[:3]
    else:
        param_names_ = param_names
    sub_dat[param_names_].plot(kind='box', by='model')
    scatter_matrix(sub_dat[param_names_])
    plt.show()

In [ ]:
# selected_params = selected_agents[param_names + ['model']]

# # par_long = pd.melt(selected_params, id_vars='model', var_name='param_name', value_name='param_value')
# # print(gg.ggplot(par_long, gg.aes('param_name', 'param_value', color='model'))
# #  + gg.geom_boxplot()
# # )  JUST TOO SLOW

# sim_params = selected_params.groupby('model').aggregate('median').reset_index()
# sim_params['alpha_high'] = np.nan
# sim_params.loc[sim_params['model'] == 'hier', 'alpha_high'] = np.median(
#     selected_params.loc[selected_params['model'] == 'hier', 'alpha_high'])
# sim_params

In [ ]:
# Create task, get numbers of trials for each phase
n_sim_per_subj, n_subj = 10, 26  # n_sim_per_sub = 20, n_subj = 31 (version3.1)  # TODO should be 1, 31-x (1 sim per person; exclude excluded subjects)
task = Task(n_subj)
n_trials, _, _, _, _ = task.get_trial_sequence(get_alien_paths(False)["human data prepr"],
                                         n_subj, n_sim_per_subj, range(n_subj),
                                         phases=("1InitialLearning", "2CloudySeason"))

n_trials_ = {'1InitialLearn': np.sum(task.phase == '1InitialLearning'),
             '2CloudySeason': np.sum(task.phase == '2CloudySeason'),
             '4RainbowSeason': 4 * n_aliens,
             '5Competition': 3}

trials = {'1InitialLearn': range(n_trials_['1InitialLearn']),
          '2CloudySeason': range(n_trials_['1InitialLearn'],
                                 n_trials_['1InitialLearn'] + n_trials_['2CloudySeason'])}

In [ ]:
model_name = 'hier'

param_names, param_ranges = get_param_names_ranges(model_name)
selected_params = selected_agents[param_names + ['model']]
sim_params = selected_params.groupby('model').aggregate('median').reset_index()
sim_params['alpha_high'] = np.nan
sim_params.loc[sim_params['model'] == 'hier', 'alpha_high'] = np.median(
    selected_params.loc[selected_params['model'] == 'hier', 'alpha_high'])

# Simulate one full dataset
## Get parameters
params_0inf = sim_params.loc[sim_params['model'] == model_name]
params_0inf = params_0inf.drop(columns=['model'])
params_0inf = params_0inf.reindex(param_names, axis=1)

params_01 = (params_0inf - param_ranges.loc[0]) / (param_ranges.loc[1] - param_ranges.loc[0])
param_values = params_01.values.flatten()

## Simulate
n_sim = n_sim_per_subj * n_subj
print("Simulating {} {} agents with parameters:\n{}\n".format(n_sim, model_name, params_0inf))
sim_summary = get_summary(
    param_values, param_ranges, n_sim, n_subj, model_name, summary_or_fulldata='summary')
sim_summary[:40], sim_summary[40:]
sim_summary = sim_summary.T
sim_summary['model'] = model_name

In [ ]:
# ## Simulate
# print("Simulating {} {} agents with parameters:\n{}\n".format(n_sim, model_name, params_0inf))
# seasons, corrects, rewards, aliens, actions, TSs = get_summary(
#     param_values, param_ranges, n_sim, n_subj, model_name, summary_or_fulldata='fulldata')

# ## Save individual agents to disk like humans
# for agentID in range(n_sim):

#     # Create pandas DataFrame
#     subj_data = pd.DataFrame()
#     subj_data["context"] = seasons[:, agentID]
#     subj_data["sad_alien"] = aliens[:, agentID]
#     subj_data["item_chosen"] = actions[:, agentID]
#     subj_data["reward"] = rewards[:, agentID]
#     subj_data["correct"] = corrects[:, agentID]
#     subj_data["trial_type"] = "feed-aliens"
#     subj_data["trial_index"] = np.arange(n_trials)
#     subj_data["subjID"] = agentID
#     subj_data["block.type"] = "normal"
#     subj_data["model_name"] = model_name
#     subj_data["phase"] = ['1InitialLearning'] * n_trials_['1InitialLearn'] + ['2CloudySeason'] * n_trials_['2CloudySeason']
#     for i, param_name in enumerate(param_names):
#         subj_data[param_name] = param_values[i]

#     # Save to disc
#     file_name = "{}/aliens_{}_ag{}_modelidx{}.csv".format(
#         full_summary_save_dir, model_name, agentID, 'median')
#     print('Saving file {0}'.format(file_name))
#     subj_data.to_csv(file_name)

## Plot bar graphs for humans and agents

In [ ]:
# Plot human behavior & selected_agents
def make_plot(hum_mean, ag_mean, hum_se=False, ag_se=False, plot_name="plot.png", ylabel="",
              hline=False, ylim=False, xlabel="", xticklabels=False, figsize=(8,4), plotted_models=models):

    if not np.any(hum_se):
        hum_se = np.zeros(len(hum_mean.T))
    if not np.any(ag_se):
        ag_se = np.zeros(3)
    if not xticklabels:
        xticklabels = ""

    fig, axes = plt.subplots(nrows=1, ncols=4, figsize=figsize)
    [ax.set_title(title) for ax, title in zip(axes, ['Humans'] + models)]

    axes[0].bar(range(len(hum_mean)), hum_mean, yerr=hum_se, tick_label=xticklabels, color='grey')
    for i_model in range(len(plotted_models)):
        axes[i_model+1].bar(range(len(hum_mean)), ag_mean[i_model], yerr=ag_se[i_model],
                          tick_label=xticklabels, color='grey')

    [ax.set_ylabel(ylabel) for ax in axes]
    [ax.set_xlabel(xlabel) for ax in axes]

    if ylim:
        [ax.set_ylim(ylim) for ax in axes]
    if hline:
        [ax.axhline(y=hline, color='black', linestyle='--') for ax in axes]

    plt.tight_layout()
    plt.savefig(plot_save_dir + plot_name)

In [ ]:
# # Get agent data
# model_idx = 0
# bar_models = ['hier', 'flat']

# selected_agents = pd.read_csv(summary_save_dir + '/selected_agents.csv', index_col=0)
# selected_agents = selected_agents.reset_index()
# ag_summary = pd.DataFrame(
#     [selected_agents.loc[selected_agents['model'] == model
#                         ].reset_index(drop=True).loc[min(model_ns[i]-1, model_idx)]
#      for i, model in zip([0, 2], bar_models)]
#     )

# save_dir = summary_save_dir + 'ag_summary_for_paper.csv'
# ag_summary.to_csv(save_dir)
# print("Saving agents used for paper plots to {}!".format(save_dir))
# ag_summary

In [ ]:
# ag_summary_rainbow = pd.DataFrame()
# ag_rainbow_dat = list()

# for model in bar_models:
#     model_dat = ag_summary.loc[ag_summary['model'] == model, RB_cols]
#     mod_ag_rainbow_dat = model_dat.values.reshape((n_aliens, n_actions))
#     mod_ag_summary_rainbow = get_summary_rainbow(n_aliens, n_seasons, mod_ag_rainbow_dat, task)
#     mod_ag_summary_rainbow = pd.DataFrame(
#         data=np.expand_dims(mod_ag_summary_rainbow, axis=0),
#         columns=RB_sum_cols)
    
#     ag_summary_rainbow = ag_summary_rainbow.append(mod_ag_summary_rainbow)
#     ag_rainbow_dat.append(mod_ag_rainbow_dat)
    
# ag_summary_rainbow['TS1minusTS2'] = ag_summary_rainbow['TS1'] - ag_summary_rainbow['TS2']
# ag_summary_rainbow['TS0minusTS2'] = ag_summary_rainbow['TS0'] - ag_summary_rainbow['TS2']
# ag_summary_rainbow

In [ ]:
sim_summary_rainbow = pd.DataFrame()
sim_rainbow_dat = list()

for model in ['hier']:
    model_dat = sim_summary.loc[sim_summary['model'] == model, RB_cols]
    mod_sim_rainbow_dat = model_dat.values.reshape((n_aliens, n_actions))
    mod_sim_summary_rainbow = get_summary_rainbow(n_aliens, n_seasons, mod_sim_rainbow_dat, task)
    mod_sim_summary_rainbow = pd.DataFrame(
        data=np.expand_dims(mod_sim_summary_rainbow, axis=0),
        columns=RB_sum_cols)
    
    sim_summary_rainbow = sim_summary_rainbow.append(mod_sim_summary_rainbow)
    sim_rainbow_dat.append(mod_sim_rainbow_dat)
    
sim_summary_rainbow['TS1minusTS2'] = sim_summary_rainbow['TS1'] - sim_summary_rainbow['TS2']
sim_summary_rainbow['TS0minusTS2'] = sim_summary_rainbow['TS0'] - sim_summary_rainbow['TS2']
sim_summary_rainbow

In [ ]:
# Plot intrusion errors
make_plot(hum_summary_initial_learn[IL_cols[3:5]].values.flatten(), sim_summary[IL_cols[3:5]].values,
          hum_summary_initial_learn[IL_cols[6:8]].values.flatten(), sim_summary[IL_cols[6:8]].values,
          plot_name='1intrusion_errors_{}.png'.format(model_idx), figsize=(8, 4),
          ylabel="Fraction (trial 1)", hline=1/3, ylim=(0, 0.5), xticklabels=['Acc.', 'Intr.err.'],
          plotted_models=['hier',])

# # Plot TS values affect performance
# make_plot(hum_summary_initial_learn[IL_cols[10:12]].values.flatten(), sim_summary[IL_cols[10:12]].values,
#           hum_summary_initial_learn[IL_cols[13:15]].values.flatten(), sim_summary[IL_cols[13:15]].values,
#           plot_name='2TS_values_perf_{}.png.'.format(model_idx), figsize=(8, 4),
#           ylabel="TS performance", hline=1/3, ylim=(1/3, 0.65), xticklabels=['TS2', 'TS1'],
#           plotted_models=['hier',])

# # Plot Savings
# make_plot(hum_summary_initial_learn[IL_cols[:3]].values.flatten(), sim_summary[IL_cols[:3]].values,
#           plot_name='5Savings.png', figsize=(12, 4),
#           ylabel="Savings", xticklabels=IL_cols[:3])

In [ ]:
# Plot TS reactivation
make_plot(hum_summary_cloudy[CL_cols[:4]].values.flatten(), sim_summary[CL_cols[:4]].values,
          hum_summary_cloudy[CL_cols[4:8]].values.flatten(), sim_summary[CL_cols[4:8]].values,
          plot_name='0TS_react_{}.png'.format(model_idx), figsize=(12, 4),
          ylabel="% correct", hline=False, ylim=(0, 0.45),
          xlabel="Trial", xticklabels=range(1, n_aliens+1),
          plotted_models=['hier'])

In [ ]:
# TS values affect preference
make_plot(hum_summary_competition[CO_cols[:2]].values.flatten(), sim_summary[CO_cols[:2]].values,
          hum_summary_competition[CO_cols[2:4]].values.flatten(), sim_summary[CO_cols[2:4]].values,
          plot_name='3TS_values_preference_{}.png'.format(model_idx), figsize=(8, 4),
          ylabel="frac. better chosen", hline=1/2, ylim=(0, 0.7), xticklabels=['Context', 'Stimulus'],
          plotted_models=['hier'])

In [ ]:
# Plot TS values affect generalization
make_plot(hum_summary_rainbow[RB_sum_cols[:4]].values.flatten(), sim_summary_rainbow[RB_sum_cols[:4]].values,
          hum_summary_rainbow[RB_sum_cols[4:8]].values.flatten(),
          plot_name='4TS_values_generalization_{}.png'.format(model_idx), figsize=(12, 4),
          ylabel='% TS selected', ylim=(0, 0.5), xticklabels=['TS3', 'TS2', 'TS1', 'No-TS'],
          plotted_models=['hier'])

In [ ]:
# # Rainbow phase correlation between human and simulated actions
# rb_cor = np.corrcoef(hum_rainbow_dat[0].flatten(), ag_rainbow_dat.astype(float).flatten())[0, 1]
# stats.pearsonr(hum_rainbow_dat[0].flatten(), ag_rainbow_dat.astype(float).flatten())

# Rainbow phase action heatmaps
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(12, 4))
# plt.suptitle('Correlation between humans & simulation: {}'.format(rb_cor.round(3)))
[ax.set_title(title) for ax, title in zip(axes, ['Humans'] + ['hier'])]
cax1 = axes[0].matshow(hum_rainbow_dat[0])
for i in range(len(['hier'])):
    axes[i+1].matshow(pd.DataFrame(ag_rainbow_dat[i]).astype(float))
[ax.set_xlabel('Action') for ax in axes]
[ax.set_ylabel('Stimulus') for ax in axes]
# fig.colorbar(cax1)

# # Show values of correct actions in rainbow phase
# correct_TS = task.TS.copy().astype(float)
# correct_TS[correct_TS == 1] = np.nan
# av_Q_correct_action = np.nanmean(correct_TS, axis=0)
# av_Q_correct_action[np.isnan(av_Q_correct_action)] = 0
# axes[1].set_title("Values correct actions")
# axes[1].matshow(av_Q_correct_action)
# for (i, j), z in np.ndenumerate(av_Q_correct_action):
#     axes[1].text(j, i, '{:0.1f}'.format(z), ha='center', va='center')
# axes[0].set_xlabel('Aliens')
# axes[0].set_ylabel('Actions')

# Old stuff

In [ ]:
def scale_parameters(parameters, param_ranges):
    
    ## Scale parameters correctly
    parameters = param_ranges.loc[0] + (param_ranges.loc[1] - param_ranges.loc[0]) * parameters

    beta_shape = (n_sim, 1)  # Q_low_sub.shape -> [n_subj, n_actions]
    beta_high_shape = (n_sim, 1)  # Q_high_sub.shape -> [n_subj, n_TS]
    forget_shape = (n_sim, 1, 1, 1)  # Q_low[0].shape -> [n_subj, n_TS, n_aliens, n_actions]
    forget_high_shape = (n_sim, 1, 1)  # -> [n_subj, n_seasons, n_TS]

    ## Parameters present in all models (flat RL, hier RL, Bayes)
    alpha = parameters['alpha'] * np.ones(n_sim)
    beta = parameters['beta'] * np.ones(beta_shape)
    forget = parameters['forget'] * np.ones(forget_shape)

    ## Deal with parameters that exist only in some models
    try:
        alpha_high = parameters['alpha_high'] * np.ones(n_sim)
    except KeyError:
        alpha_high = np.zeros(n_sim)
    try:
        beta_high = parameters['beta_high'] * np.ones(beta_high_shape)
    except KeyError:
        beta_high = np.zeros(beta_high_shape)
    try:
        forget_high = parameters['forget_high'] * np.ones(forget_high_shape)
    except KeyError:
        forget_high = np.zeros(forget_high_shape)

    return alpha, beta, forget, alpha_high, beta_high, forget_high

In [ ]:
def simulate_initial_learn(n_trials, n_sim, n_TS, n_aliens, n_actions, n_seasons,
                           alpha, beta, forget, alpha_high, beta_high, forget_high,
                           alien_initial_Q, trials, task):
    
    ## Set up data storage
    seasons = np.zeros([n_trials, n_sim], dtype=int)
    corrects = np.zeros([n_trials, n_sim])
    rewards = np.zeros([n_trials, n_sim])
    aliens = np.zeros([n_trials, n_sim], dtype=int)
    actions = np.zeros([n_trials, n_sim], dtype=int)

    ## Inialize Q-values
    Q_low = alien_initial_Q * np.ones([n_sim, n_TS, n_aliens, n_actions])
    Q_high = alien_initial_Q * np.ones([n_sim, n_seasons, n_TS])

    ## Simulate behavior
    for trial in trials['1InitialLearn']:

        ### Observe stimuli
        season, alien = task.present_stimulus(trial)

        ### Select action & update Q-values
        [Q_low, Q_high, TS, action, correct, reward, p_low] =\
            update_Qs_sim(season, alien,
                          Q_low, Q_high,
                          beta, beta_high, alpha, alpha_high, forget, forget_high,
                          n_sim, n_actions, n_TS, task, alien_initial_Q, model_name)

        ### Store trial data
        seasons[trial] = season
        corrects[trial] = correct
        rewards[trial] = reward
        aliens[trial] = alien
        actions[trial] = action

    return Q_low, Q_high, seasons, corrects, rewards, aliens, actions, season

In [ ]:
def simulate_cloudy(trials, season, task):

    for trial in trials['2CloudySeason']:

        ## Observe trial stimuli
        old_season = season.copy()
        season, alien = task.present_stimulus(trial)

        ## Season switches
        if trial == list(trials['2CloudySeason'])[0]:
            season_switches = np.ones(n_sim, dtype=bool)
        else:
            season_switches = season != old_season

        ## Reset Q-values after season switches
        if (model_name == 'hier') or (model_name == 'Bayes'):
            Q_high[season_switches] = alien_initial_Q  # re-start search for the right TS when season changes
        elif model_name == 'flat':
            Q_low[season_switches] = alien_initial_Q  # re-learn a new policy from scratch when season changes
        else:
            raise(NameError, 'Model_name must be "flat", "hier", or "Bayes".')

        ## Update Q-values
        [Q_low, Q_high, TS, action, correct, reward, p_low] =\
            update_Qs_sim(0 * season, alien,
                          Q_low, Q_high,
                          beta, beta_high, alpha, alpha_high, forget, forget_high,
                          n_sim, n_actions, n_TS, task, alien_initial_Q, model_name)

        ## Store trial data
        seasons[trial] = season
        corrects[trial] = correct
        rewards[trial] = reward
        aliens[trial] = alien
        actions[trial] = action

    return seasons, corrects, aliens, actions

# Debugging

In [ ]:
params = np.random.rand(len(param_names))
# params[-1] = 0
# params[-2] = 0
final_Q_low, final_Q_high, Q_high_mem = get_summary(
    params, param_ranges, n_sim, n_subj, model_name, summary_or_fulldata='summary')

np.mean(np.mean(final_Q_high, axis=0), axis=1)
final_Q_high
# Q_high_mem

In [ ]:

# seasons, corrects, rewards, aliens, actions, TSs = get_summary(
#     params, param_ranges, n_sim, n_subj, model_name, summary_or_fulldata='fulldata')

# # Subset cloudy season
# phase_initial='CL'
# seasons, corrects, aliens, actions, TSs = seasons[trials['2CloudySeason']], corrects[trials['2CloudySeason']], aliens[trials['2CloudySeason']], actions[trials['2CloudySeason']], TSs[trials['2CloudySeason']]
    
# # Get season changes
# season_changes = np.array([seasons[i, 0] != seasons[i + 1, 0] for i in range(len(seasons)-1)])
# season_changes = np.insert(season_changes, 0, False)

# # Get first four trials after context switch
# alien_sum = np.array([i // 4 for i in range(len(seasons))])
# first_trials = alien_sum[season_changes]
# first_trial_mask = np.array([np.any(alien_trial == first_trials) for alien_trial in alien_sum])

# # Subset data
# first_aliens = aliens[first_trial_mask]
# first_actions = actions[first_trial_mask]
# first_TSs = TSs[first_trial_mask]

# # Get chosen TS
# key = ('TS0', 'TS1', 'TS2', 'TS02', 'TS12', 'None')
# chosen_TS_ii = [[get_chosen_TS(alien, action, task.TS, key) for alien, action in zip(first_alien, first_action)]
#              for first_alien, first_action in zip(first_aliens, first_actions)]

# # Format result
# chosen_TS_indiv = np.mean(np.array(chosen_TS_ii), axis=0)  # average over trials to get one row per participant
# chosen_TS = pd.DataFrame(data=np.mean(chosen_TS_indiv, axis=0),
#                             index=['{}_first_{}'.format(phase_initial, k) for k in key])
# chosen_TS_se = pd.DataFrame(data=np.std(chosen_TS_indiv, axis=0),
#                                index=['{}_first_{}_se'.format(phase_initial, k) for k in key])

subj = 0
'true TS:', task.TS, \
'aliens:', np.array([trial[subj] for trial in first_aliens]), \
'actions:', np.array([trial[subj] for trial in first_actions]), \
'inferred TS:', [trial[subj] for trial in chosen_TS_ii], \
'chosen TS:', first_TSs[:, subj]

# from collections import Counter
# Counter(list(first_TSs.flatten()))

In [ ]:
task.TS

In [ ]:
# print(model_name)
# phase = '1InitialLearn'  # '2CloudySeason', '1InitialLearn'

# params = np.random.rand(len(param_names))
# seasons, corrects, rewards, aliens, actions, TSs = get_summary(
#             params, param_ranges, n_sim, n_subj, model_name, summary_or_fulldata='fulldata')

# # Subset data
# seasons, corrects, aliens, actions, TSs = seasons[trials[phase]], corrects[trials[phase]], aliens[trials[phase]], actions[trials[phase]], TSs[trials[phase]]

# # Set up regression model correct ~ action_values + TS_values
# subj = 0
# action_values, TS_values = get_action_TS_values(seasons[:, subj], aliens[:, subj], task.TS)
# action_values[:10], TS_values[:10], seasons[:, subj][:10], aliens[:, subj][:10]

c = corrects.astype(int)
Qa = np.array([get_action_values(seasons[trial], aliens[trial], task.TS) for trial in range(len(seasons))])  # trial x subj
Qts = np.array([get_TS_values(seasons[trial], task.TS) for trial in range(len(seasons))])  # trial x subj
Qa, Qts, seasons, aliens

c.shape, Qa.shape, Qts.shape

c_subj = c[:, subj]
Qa_subj = Qa[:, subj]
Qts_subj = Qts[:, subj]
c_subj, Qa_subj, Qts_subj
c_subj.reshape(-1, 1)

# Qa_subj.shape, c_subj.shape, 
X_subj = np.array([Qa_subj, Qts_subj]).T
y_subj = c[:, subj]

X_subj, y_subj

regr = LogisticRegression()
regr.fit(X_subj, y_subj)
regr.coef_

In [ ]:
seasons.shape

In [ ]:
print(model_name)
params = np.random.rand(len(param_names))
seasons, corrects, rewards, aliens, actions, TSs = get_summary(
            params, param_ranges, n_sim, n_subj, model_name, summary_or_fulldata='fulldata')

# Run regression model correct ~ action_values + TS_values
regr_phases = ['1InitialLearn', '2CloudySeason']

regr_coeffs = []
for phase in regr_phases:

    # Get data
    seasons_ph, corrects_ph, aliens_ph, actions_ph, TSs_ph = seasons[trials[phase]], corrects[trials[phase]], aliens[trials[phase]], actions[trials[phase]], TSs[trials[phase]]
    c = corrects_ph.astype(int)
    Qa = np.array([get_action_values(seasons_ph[trial], aliens_ph[trial], task.TS) for trial in range(len(seasons_ph))])  # trial x subj
    Qts = np.array([get_TS_values(seasons_ph[trial], task.TS) for trial in range(len(seasons_ph))])  # trial x subj

    # Run model for each agent
    coefs = []
    for subj in range(c.shape[1]):
        c_subj = c[:, subj]
        Qa_subj = Qa[:, subj]
        Qts_subj = Qts[:, subj]

        X_subj = np.array([Qa_subj, Qts_subj]).T
        y_subj = c[:, subj]

        X_subj, y_subj

        regr = LogisticRegression(solver='lbfgs')
        regr.fit(X_subj, y_subj)
        coefs += [regr.coef_.flatten()]
        
    coefs = pd.DataFrame(
        data=np.concatenate([np.mean(np.array(coefs), axis=0), np.std(np.array(coefs), axis=0)]),
        index=['{}_{}{}'.format(q, p, s) for q, p, s in zip(2 * ['Qa', 'Qts'], 4 * [phase], 2 * ['_mean'] + 2 * ['_se'])])
    regr_coeffs += [coefs]

pd.concat(regr_coeffs, axis=0)

In [ ]:
['{}_{}{}'.format(q, p, s) for q, p, s in zip(2 * ['Qa', 'Qts'], 4 * [phase], 2 * [''] + 2 * ['_se'])]

In [ ]:

mean_coefs = np.mean(np.array(coefs), axis=0)
std_coefs = np.std(np.array(coefs), axis=0)

np.concatenate([mean_coefs, std_coefs])

In [ ]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
X, y = load_iris(return_X_y=True)
clf = LogisticRegression(random_state=0).fit(X, y)
clf.predict(X[:2, :])

clf.predict_proba(X[:2, :])

clf.score(X, y)

X.shape, y.shape
X, y

In [ ]:
params = np.random.rand(len(param_names))
get_summary(
            params, param_ranges, n_sim, n_subj, model_name, summary_or_fulldata='summary')

In [ ]:
def make_diff_hist(all_summaries, col_names, model1_name, model2_name, plot_name='test.png', n_bins=50):

    all_diff_dat = pd.DataFrame()

    for col_name in col_names:
        dat1 = all_summaries.loc[all_summaries.model==model1_name, col_name].values
        dat2 = all_summaries.loc[all_summaries.model==model2_name, col_name].values

        diff_dat = get_difference_data(dat1, dat2, n_bins=n_bins)
        diff_dat['measure'] = col_name

        all_diff_dat = all_diff_dat.append(diff_dat)

    g = (gg.ggplot(all_diff_dat, gg.aes('bin', 'diff', color='measure'))
     + gg.geom_line()
     + gg.xlab(col_name)
     + gg.ylab('Density {} minus {}'.format(model2_name, model1_name))
#      + gg.coord_cartesian(xlim=xlim, ylim=ylim)
    )
    save_path = os.path.join(plot_save_dir, plot_name)
    print("Saving to {}.".format(save_path))
    g.draw()
    g.save(save_path)
    
# Example use
for phase in ['1InitialLearn', '2CloudySeason']:
    make_diff_hist(all_summaries,
                   ['Qts_{}_mean'.format(phase), 'Qa_{}_mean'.format(phase)],
                   'flat', 'hier')

In [ ]:
# Reusable histogram function for all histogram plots
def make_histogram(sim_dat, hum_dat, columns,
                   xlabels=False, ylabel="Density", xlim=False, ylim=False, vline=False, yscale_log=False, scale_data=1, plot_name=''):
    
    nrows = max(len(columns), 2)
    fig, axes = plt.subplots(nrows=nrows, figsize=(6, nrows * 2))
    [ax.set_ylabel(ylabel) for ax in axes]
    
    if not xlabels:
        xlabels = columns
    
    for i, (effect, xlabel) in enumerate(zip(columns, xlabels)):
        for model in models:
            dat = sim_dat.loc[sim_dat['model'] == model]
            sns.distplot(scale_data*dat[effect], kde=True, hist=False, label=model, ax=axes[i], bins=19)
        if np.any(hum_dat):
            axes[i].axvline(x=scale_data*hum_dat[effect].values, color='m', linestyle='-')
        axes[i].set_xlabel(xlabel)
    
    if vline:
        [ax.axvline(x=vline, color='grey', linestyle='--') for ax in axes]
    if xlim:
        [ax.set_xlim(xlim) for ax in axes]
    if ylim:
        [ax.set_ylim(ylim) for ax in axes]
    if yscale_log:
        [ax.set_yscale('log') for ax in axes]
    
    plt.legend()
    plt.tight_layout()
    figure_path = os.path.join(plot_save_dir, plot_name)
    plt.savefig(figure_path)
    print("Saved figure to {}".format(figure_path))

def get_means_sds(sim_dat, hum_dat, column, t_compare_value=0):
    
    means_sds = sim_dat.groupby("model")[column].agg(
        {'mean': 'mean',
         'std': 'std',
         'lik': lambda x: np.mean(x > hum_dat[column][0]),
         'lik2': lambda x: np.mean(x < hum_dat[column][0])})
    
    ttests = sim_dat.groupby('model')[column].agg(stats.ttest_1samp, t_compare_value)
    
    return means_sds, ttests

In [ ]:
# # Intrusion error heatmap (prev_TS)
# fig = plt.figure(figsize=(10, 5))
# for i, effect in enumerate(['IL_acc_prev_TS', 'IL_acc_current_TS']):

#     # Subset data with human-like behavior
#     dat = all_summaries.loc[(all_summaries['model'] == 'hier') &
#                             (all_summaries['IL_acc_prev_TS'] > 0.55)]

#     # Plot raw parameters
#     ax = fig.add_subplot(2, 3, (i*3)+1, projection='3d')
#     ax.set_title(effect)
#     ax.scatter(dat['alpha'], dat['beta'], dat['forget'], c=dat[effect], label=dat[effect], marker='.')
#     ax.set_xlabel('alpha')
#     ax.set_ylabel('beta')
#     ax.set_zlabel('forget')

#     ax = fig.add_subplot(2, 3, (i*3)+2, projection='3d')
#     ax.set_title(effect)
#     ax.scatter(dat['alpha_high'], dat['beta_high'], dat['forget_high'], c=dat[effect], label=dat[effect], marker='.')
#     ax.set_xlabel('alpha_high')
#     ax.set_ylabel('beta_high')
#     ax.set_zlabel('forget_high')

#     if do_isomap:
#         # Dimensionality reduction
#         n_neighbors = 12
#         n_components = 3
#         params_norm = preprocessing.StandardScaler().fit_transform(dat[param_names])  # standardize data
#         Y = manifold.Isomap(n_neighbors, n_components).fit_transform(params_norm)  # apply Isomap

#         # Plot dimensionality-reduced parameters
#         ax = fig.add_subplot(2, 3, (i*3)+3, projection='3d')
#         ax.set_title("Isomap " + effect)
#         ax.scatter(Y[:, 0], Y[:, 1], Y[:, 2], c=dat[effect], marker='.')

#     plt.tight_layout()
# plt.show()

In [ ]:
# # Calculate Isomap & Co. and visualize
# params_norm = preprocessing.StandardScaler().fit_transform(dat[param_names])  # standardize data
# effect = dat['IL_acc_prev_TS']
# fig = plt.figure(figsize=(15, 5))
#
# i = 1
# for n_components in range(2, 6, 2):
#     for n_neighbors in range(2, 20, 3):
#
#         # Reduce dimensionality
#         # Y = decomposition.PCA(n_components).fit_transform(params_norm)
#         # mds = manifold.MDS(n_neighbors, max_iter=100, n_init=1)
#         # Y = mds.fit_transform(params_norm)
#         # Y = manifold.LocallyLinearEmbedding(n_neighbors, n_components, eigen_solver='auto',
#         #                                     method='standard').fit_transform(params_norm)
#         # Y = manifold.LocallyLinearEmbedding(n_neighbors, n_components, eigen_solver='auto',  # n_neighbors >= 6
#         #                                     method='modified').fit_transform(params_norm)
#         Y = manifold.Isomap(n_neighbors, n_components).fit_transform(params_norm)
#
#         # Plot
#         ax = fig.add_subplot(2, 6, i)
#         plt.scatter(Y[:, 0], Y[:, 1], c=effect)
#         plt.title('{0} comp., {1} neigh.'.format(n_components, n_neighbors))
#         i += 1
# plt.tight_layout()
# plt.show()

# Intrusion error heatmap (other TS)
fig = plt.figure()
for i, effect in enumerate(['IL_acc_other_TS', 'IL_acc_current_TS']):

    dat = all_summaries.loc[(all_summaries['model'] == 'hier') &
                            (all_summaries['IL_acc_other_TS'] > 1/3)]

    ax = fig.add_subplot(2, 2, i+1, projection='3d')
    ax.set_title(effect)
    ax.scatter(dat['alpha'], dat['beta'], dat['forget'], c=dat[effect], label=dat[effect], marker='.')
    ax.set_xlabel('alpha')
    ax.set_ylabel('beta')
    ax.set_zlabel('forget')

    ax = fig.add_subplot(2, 2, i+3, projection='3d')
    ax.set_title(effect)
    ax.scatter(dat['alpha_high'], dat['beta_high'], dat['forget_high'], c=dat[effect], label=dat[effect], marker='.')
    ax.set_xlabel('alpha_high')
    ax.set_ylabel('beta_high')
    ax.set_zlabel('forget_high')
plt.tight_layout()

# Savings heatmap
fig = plt.figure()
for i, effect in enumerate(['IL_saving_av', 'IL_saving_first_trial', 'IL_saving_last_trial', 'IL_saving_last_minus_first']):

    dat = all_summaries.loc[(all_summaries['model'] == 'hier') &
                            (all_summaries['IL_saving_last_trial'] > 0.01) & (all_summaries['IL_saving_first_trial'] > 0.08)]

    ax = fig.add_subplot(2, 4, i+1, projection='3d')
    ax.set_title(effect)
    ax.scatter(dat['alpha'], dat['beta'], dat['forget'], c=dat[effect], label=dat[effect], marker='.')
    ax.set_xlabel('alpha')
    ax.set_ylabel('beta')
    ax.set_zlabel('forget')

    ax = fig.add_subplot(2, 4, i+5, projection='3d')
    ax.set_title(effect)
    ax.scatter(dat['alpha_high'], dat['beta_high'], dat['forget_high'], c=dat[effect], label=dat[effect], marker='.')
    ax.set_xlabel('alpha_high')
    ax.set_ylabel('beta_high')
    ax.set_zlabel('forget_high')
plt.tight_layout()

# Cloudy heatmap
fig = plt.figure()
for i, effect in enumerate(['CL_acc_trial2', 'CL_acc_trial3']):

    dat = all_summaries.loc[(all_summaries['model'] == 'hier') &
                            (all_summaries['CL_acc_trial2'] > 0.4) & (all_summaries['CL_acc_trial3'] > 0.4)]

    ax = fig.add_subplot(2, 2, i+1, projection='3d')
    ax.set_title(effect)
    ax.scatter(dat['alpha'], dat['beta'], dat['forget'], c=dat[effect], label=dat[effect], marker='.')
    ax.set_xlabel('alpha')
    ax.set_ylabel('beta')
    ax.set_zlabel('forget')

    ax = fig.add_subplot(2, 2, i+3, projection='3d')
    ax.set_title(effect)
    ax.scatter(dat['alpha_high'], dat['beta_high'], dat['forget_high'], c=dat[effect], label=dat[effect], marker='.')
    ax.set_xlabel('alpha_high')
    ax.set_ylabel('beta_high')
    ax.set_zlabel('forget_high')
plt.tight_layout()

# Competition heatmap
fig = plt.figure()
for i, effect in enumerate(['CO_acc_season', 'CO_acc_season_alien']):

    dat = all_summaries.loc[(all_summaries['model'] == 'hier') &
                            (all_summaries['CO_acc_season'] > 0.65)]  #  &(all_summaries['CO_acc_season_alien'] > 0.60)

    ax = fig.add_subplot(2, 2, i+1, projection='3d')
    ax.set_title(effect)
    ax.scatter(dat['alpha'], dat['beta'], dat['forget'], c=dat[effect], label=dat[effect], marker='.')
    ax.set_xlabel('alpha')
    ax.set_ylabel('beta')
    ax.set_zlabel('forget')

    ax = fig.add_subplot(2, 2, i+3, projection='3d')
    ax.set_title(effect)
    ax.scatter(dat['alpha_high'], dat['beta_high'], dat['forget_high'], c=dat[effect], label=dat[effect], marker='.')
    ax.set_xlabel('alpha_high')
    ax.set_ylabel('beta_high')
    ax.set_zlabel('forget_high')
plt.tight_layout()

# Rainbow heatmap
fig = plt.figure()
for i, effect in enumerate(['RB_choices_TS0', 'RB_choices_TS1', 'RB_choices_TS2']):

    dat = all_summaries.loc[(all_summaries['model'] == 'hier') &
                            (all_summaries['RB_choices_TS0'] > 1300) & (all_summaries['RB_choices_TS1'] > 1000)]

    ax = fig.add_subplot(2, 3, i+1, projection='3d')
    ax.set_title(effect)
    ax.scatter(dat['alpha'], dat['beta'], dat['forget'], c=dat[effect], label=dat[effect], marker='.')
    ax.set_xlabel('alpha')
    ax.set_ylabel('beta')
    ax.set_zlabel('forget')

    ax = fig.add_subplot(2, 3, i+4, projection='3d')
    ax.set_title(effect)
    ax.scatter(dat['alpha_high'], dat['beta_high'], dat['forget_high'], c=dat[effect], label=dat[effect], marker='.')
    ax.set_xlabel('alpha_high')
    ax.set_ylabel('beta_high')
    ax.set_zlabel('forget_high')
plt.tight_layout()

# Plot overall heatmaps -> systematicity between parameters & effects?
for effect in summary_dat_cols:
    fig = plt.figure()
    plt.title(effect)
    for i, model in enumerate(models):
        dat = all_summaries.loc[all_summaries['model'] == model][:1000].copy()

        ax = fig.add_subplot(2, 2, i+1, projection='3d')
        ax.scatter(dat['alpha'], dat['beta'], dat['forget'], c=dat[effect], label=dat[effect], marker='.')
        ax.set_title(model)
        ax.set_xlabel('alpha')
        ax.set_ylabel('beta')
        ax.set_zlabel('forget')

        ax = fig.add_subplot(2, 2, i+3, projection='3d')
        ax.scatter(dat['alpha_high'], dat['beta_high'], dat['forget_high'], c=dat[effect], label=dat[effect], marker='.')
        ax.set_xlabel('alpha_high')
        ax.set_ylabel('beta_high')
        ax.set_zlabel('forget_high')
    plt.tight_layout()

# Plot average effects for hierarchical and flat
fig = plt.figure()
ax = fig.add_subplot(111)

for i, model in enumerate(models):
    dat = all_summaries.loc[all_summaries['model'] == model].copy()
    RB_effects = [effect for effect in all_summaries.columns.values if 'RB' in effect]

    dat[RB_effects] /= 1000
    dat['beta'] /= 10
    x = np.arange(len(summary_dat_cols)) + 0.2 * (2*i-1)
    y = np.mean(dat[summary_dat_cols], axis=0)
    yerr = np.std(dat[summary_dat_cols], axis=0) / np.sqrt(len(summary_dat_cols))

    ax.bar(x, y.values, 0.4, yerr=yerr, label=model)
ax.set_xticks(np.arange(len(summary_dat_cols)))
ax.set_xticklabels(summary_dat_cols)
plt.xticks(rotation=20)
plt.ylabel('Effect (RB /= 1000)')
plt.legend()

In [ ]:
# # Double-check selected hier-agent summary
# # summary_save_dir_ag = os.path.join(get_alien_paths(run_on_cluster)['fitting results'], 'SummariesInsteadOfFitting')  # Where will the simulated summary data be saved and read from?
# params = pd.read_csv(old_summary_save_dir + '/ag_summary_for_paper.csv', index_col=0).loc[param_names]
# params_01 = params.values.flatten() / (param_ranges.loc[1] - param_ranges.loc[0]) - param_ranges.loc[0] / (
#             param_ranges.loc[1] - param_ranges.loc[0])
# # params_back = param_ranges.loc[0] + (param_ranges.loc[1] - param_ranges.loc[0]) * params_01  # making sure i'm getting the right parameters back after transformation!

# summary = get_summary(params_01.values.flatten(), param_ranges, n_sim, n_subj)
# summary = pd.DataFrame(summary, index=summary_dat_cols).transpose()

# summary  # check that it agrees with what we have in the paper! (This is just to double-check that I'm using the right parameters for the simulations!

In [ ]:
hum_summary_initial_learn['source'] = 'human'
hum_summary_initial_learn

In [ ]:
sim_summary['source'] = 'simulation'
sim_summary

In [ ]:
hum_and_sim = hum_summary_initial_learn.append(sim_summary, sort=False)
hum_and_sim

In [ ]:
cols = ['IL_acc_prev_TS', 'IL_acc_current_TS']
intr_dat = hum_and_sim[cols + ['{}_se'.format(col) for col in cols] + ['source']]
intr_long = pd.melt(intr_dat, id_vars='source')

(gg.ggplot(intrerr_dat, gg.aes('')))

# intrerr_dat
intr_long

In [ ]:
hum_and_sim_long = pd.melt(hum_and_sim, id_vars='source')
(gg.ggplot(hum_and_sim_long, gg.aes('variable', 'value', color='source'))
 + gg.geom_point()
)